# [Issue# 1897 Additional Visuals for PUC Analysis](https://github.com/cal-itp/data-analyses/issues/1897)

Received list of transit operators cohorts that may be exempt from efficiency reporting, per PUC 99314.11, .6 and .7. 
- create visuals based on grouping set by list
- recreate visuals based on previous notebook work

## [99314.6](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.6.&lawCode=PUC)
>`funds shall be allocated for operating or capital purpose` pursuant to Sections 99313 and 99314 to an operator `if the operator meets either of the following efficiency standards`:
>- (A) `The operator shall receive its entire allocation`, and any or all of this allocation may be used for operating purposes, if the operator’s `total operating cost per revenue vehicle hour` in the latest year for which audited data are available `does not exceed the sum of the preceding year’s total operating cost per revenue vehicle hour and an amount equal to the product of the percentage change in the Consumer Price Index for the same period multiplied by the preceding year’s total operating cost per revenue vehicle hour.`
>- (B) The operator shall receive its entire allocation, and any or all of this allocation may be used for operating purposes, `if the operator’s average total operating cost per revenue vehicle hour` in the latest three years for which audited data are available `does not exceed the sum of the average of the total operating cost per revenue vehicle hour in the three years preceding the latest year for which audited data are available and an amount equal to the product of the average percentage change in the Consumer Price Index for the same period multiplied by the average total operating cost per revenue vehicle hour in the same three years`.
## [99314.7 (mainly MTC specific)](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PUC&sectionNum=99314.7.)
>the `Metropolitan Transportation Commission` shall apply the following eligibility standards to the operators within the region subject to its jurisdiction:

# [99314.11](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.11.&nodeTreePath=17.11.2.8&lawCode=PUC)
>`Sections 99314.6 and 99314.7 do not apply to an operator for a fiscal year in which the operator expended from local funding an amount for transit operations not less than the amount the operator expended from local funding for transit operations during the 2018–19 fiscal year.` As used in this subdivision, “local funding” means any nonstate grant funds or other revenues generated by, earned by, or distributed to, an operator.

Meaning, if a transit operator spent local funds >= the local funds spent during FY 2018-2019, they are exempt from meeting efficiency standards(?)

## Data Exploration

### Categorical variaables
- Underlying metric
  - Farebox Recovery Ratio
  - Local funding expended
- area type
  - urban
  - rural
- cohorts
  - A
  - B
  - C
- NTD metric
  - UPT
  - PMT
  - VRH
- year
  - 2019
  - 2020
  - 2021
  - 2022
  - 2023
  - 2024

## analyses should be split by underlying metric
resulting groups are:
1. Farebox Recovery ratio
    - urban
        - cohorts
        - ntd metric
        - year
    - rural
        - cohorts
        - ntd metrics
        - year
2. Local funding expended
    - urban
        - cohorts
        - ntd metric
        - year
    - rural
        - cohorts
        - ntd metrics
        - year

## 



In [1]:
import pandas as pd
import altair as alt
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas
from calitp_data_analysis.sql import get_engine, to_snakecase, query_sql

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

@cache
def gcs_pandas():
    return GCSPandas()

gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"

# Read in cohort list data

In [ ]:
# cohort_data = gcs_pandas().read_csv(f"{gcs_path}fbr_local_funding_by_cohorts_2019-2024_compiled.csv")

# cohort_data.columns = cohort_data.columns.str.lower()
# cohort_data["ntd_id"] = cohort_data["ntd_id"].astype("str")

# display(
#     cohort_data.info(),
#     cohort_data.head(),
#     cohort_data.value_counts(
#     subset=["urban_rural","metric","cohort","year"]
#     )
# )

# Read in yes/no list data

In [ ]:
yes_no_data = gcs_pandas().read_csv(f"{gcs_path}cs_sco_yes_no_fbr_funding_2019-2024.csv")

yes_no_data.columns = yes_no_data.columns.str.lower()
yes_no_data[["year","ntd_id"]] = yes_no_data[["year","ntd_id"]].astype("str")
yes_no_data = yes_no_data.rename(columns={"requirement_flag":"requirement_met_flag"})

# yes_no_data["requirement_met_flag"] = yes_no_data["requirement_met_flag"].str.lower().map({
#     "yes":True,
#     "no":False
# })
display(
    yes_no_data.info(),
    yes_no_data.head(3)
)

In [ ]:
yes_no_data.iloc[1193:1196]

## Do yes/no actually match up with the metric value?
if Urban, FBR >=20 == pass
if Urban, FBR <20 == fail
if Rural, FBR >= 10 == pass
if Rural, FBR <10 == fail

If Urban, Pct_Change_vs_2019 >=0 == Pass
if Urban, Pct_Change_vs_2019 <0 == fail
if Rural, Pct_Change_vs_2019 >=0 == Pass
if Rural, Pct_Change_vs_2019 <0 == fail

In [ ]:
# Urban FBR check
display(
        yes_no_data[
        (yes_no_data["metric_short"] == "FBR")
        & (yes_no_data["area_type"]=="Urban")
        & (yes_no_data["metric_value"]>=20)
    ].value_counts(subset=["year","requirement_met_flag"]), # expect only True values
    
    yes_no_data[
        (yes_no_data["metric_short"] == "FBR")
        & (yes_no_data["area_type"]=="Urban")
        & (yes_no_data["metric_value"]<20)
    ].value_counts(subset=["year","requirement_met_flag"]) # expect only False values 
)

In [ ]:
# Rural FBR Check
display(
        yes_no_data[
        (yes_no_data["metric_short"] == "FBR")
        & (yes_no_data["area_type"]=="Rural")
        & (yes_no_data["metric_value"]>=10)
    ].value_counts(subset=["year","requirement_met_flag"]), # expect only True values, GTG
    
    yes_no_data[
        (yes_no_data["metric_short"] == "FBR")
        & (yes_no_data["area_type"]=="Rural")
        & (yes_no_data["metric_value"]<10)
    ].value_counts(subset=["year","requirement_met_flag"]) # expect only False values
)

In [ ]:
# Urban funding check
display(
    yes_no_data[
        (yes_no_data["metric_short"] == "Pct_Change_vs_2019")
        & (yes_no_data["area_type"]=="Urban")
        & (yes_no_data["metric_value"]>=0)
    ].value_counts(subset=["year","requirement_met_flag"]), # expect only True values
    
    yes_no_data[
        (yes_no_data["metric_short"] == "Pct_Change_vs_2019")
        & (yes_no_data["area_type"]=="Urban")
        & (yes_no_data["metric_value"]<0)
    ].value_counts(subset=["year","requirement_met_flag"]) # expect only False falues 
)

In [ ]:
# Rural funding check
display(
    yes_no_data[
        (yes_no_data["metric_short"] == "Pct_Change_vs_2019")
        & (yes_no_data["area_type"]=="Rural")
        & (yes_no_data["metric_value"]>=0)
    ].value_counts(subset=["year","requirement_met_flag"]),
    
    yes_no_data[
        (yes_no_data["metric_short"] == "Pct_Change_vs_2019")
        & (yes_no_data["area_type"]=="Rural")
        & (yes_no_data["metric_value"]<0)
    ].value_counts(subset=["year","requirement_met_flag"])
)

## Yeah this all checks out. GTG

# Read in analysis data from prev notebook

In [ ]:

# ntd_name = "puc_analysis_data.parquet"
# ntd_analysis_data = gcs_pandas().read_parquet(f"{gcs_path}{ntd_name}")

# display(
#     ntd_analysis_data.info(),
#     ntd_analysis_data["year"].unique()
# )

# May need to requery this data to include 2024
is 2024 NTD data in the warehouse now? copy pasted from initial puc analysis notebook.

In [ ]:
metric_list = [
    "pmt",
    "upt",
    "vrh",
    # "opexp_total" # not needed for this project
]

# empty list for appending DFs
df_list = []

# loop to query pmt, upt and vrh from 2018 to 2024
for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type,
          SUM({metric}) AS total_{metric},
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year BETWEEN 2018 AND 2024
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type
        """
        # create df
        metric = query_sql(query, as_df=True)

        # append df to list
        df_list.append(metric)

# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

display( 
    ntd_upt.head(3)
)

## merge all the metrics together

In [ ]:
merge_on_col = [
    "ntd_id",
    "year",
    "source_agency",
    "agency_status",
    "primary_uza_name",
    "uza_population",
    "uza_area_sq_miles",
    "mode",
    "type_of_service",
    "reporter_type",
]

merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

ntd_metrics_merge.head(3)

## get districts for ntd ID
- Do i still need district data for this specific analysis?

In [ ]:
for metric in metric_list:
        query = f"""
        SELECT
          `mart_transit_database.dim_organizations`.`key` AS `key`,
          `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
          `mart_transit_database.dim_organizations`.`name` AS `name`,
          `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
          `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
          `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
        FROM
          `mart_transit_database.dim_organizations`

        LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
          LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
        WHERE
          (
            `mart_transit_database.dim_organizations`.`_is_current` = TRUE
          )

           AND (
            `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
          )
          AND (
            (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
            )

            OR (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
            )
          )
          AND (
            `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
          )
          AND (
            `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
          )
        """
        # create df
        ntd_id_x_district = query_sql(query, as_df=True)
        
ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

ntd_id_x_district.head()

## merge the ntd metrics with Caltrans Districts

In [ ]:
ntd_metrics_merge = ntd_metrics_merge.merge(
    ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
    left_on = "ntd_id",
    right_on = "ntd_id_2022",
    how="inner",
    indicator=True
)
ntd_metrics_merge["year"] = ntd_metrics_merge["year"].astype("str")
ntd_metrics_merge.head()

# merge ntd metrics with ~~cohort data~~ yes/no data
- merge on ntd_id
- are there any unmerged rows?

In [ ]:
ntd_metrics_merge.dtypes

In [ ]:
ntd_yes_no_merge = ntd_metrics_merge.drop(columns="_merge").merge(
    yes_no_data,
    left_on = ["ntd_id","year"],
    right_on = ["ntd_id","year"],
    indicator= True,
)

# any unmerged rows? NONE
ntd_yes_no_merge["_merge"].value_counts()

In [ ]:
# Sanity check
# pick up a couple of NTD ID, see if the merge data tracks with the cohort data
sample_ids = ntd_yes_no_merge["ntd_id"].sample(3).to_list()
keep_cols=[
    "ntd_id",
    "source_agency",
    "mode",
    "type_of_service",
    "total_vrh",
    "total_pmt",
    "total_upt",
    "area_type",
    "metric",
    "year",
    "requirement",
    "requirement_met_flag"
]

# for sample_id in sample_ids:
#     display(
#         f"Sameple NTD ID: {sample_id}",
#         "cohort data",
#         yes_no_data[
#             (yes_no_data["ntd_id"]== sample_id)
#             & (yes_no_data["year"].isin(["2023","2024"]))
#             ].sort_values(by=["area_type","metric","year"]).head(5),
#         "merge table",
#         ntd_yes_no_merge[
#             (ntd_yes_no_merge["ntd_id"]== sample_id)
#             & (ntd_yes_no_merge["year"].isin(["2023","2024"]))
#             ][keep_cols].sort_values(by=["area_type","metric","year"]),
        
#     )

# cohort data matches, 
# looks a little weird since the ntd metrics is per mode and TOS. the cohort data becomes categorical. GTG

# Save merged cohort data

In [ ]:
# cort_merge_filname = "ntd_cohort_data_2026-01-26.parquet"
# gcs_pandas().data_frame_to_parquet(ntd_cohort_merge,f"{gcs_path}{cort_merge_filname}")

# Save merged  yes/no data

In [2]:
yes_no_merge_filname = "ntd_yes_no_data_2026-01-29.parquet"
# gcs_pandas().data_frame_to_parquet(ntd_yes_no_merge,f"{gcs_path}{yes_no_merge_filname}")

# Read in merged ~~cohort~~ yes/no data from GCS

In [3]:
# ntd_cohort_merge = gcs_pandas().read_parquet(f"{gcs_path}{cort_merge_filname}")

ntd_yes_no_merge = gcs_pandas().read_parquet(f"{gcs_path}{yes_no_merge_filname}")

display(
    ntd_yes_no_merge.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4965 entries, 0 to 4964
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ntd_id                4965 non-null   object  
 1   source_agency         4965 non-null   object  
 2   agency_status         4965 non-null   object  
 3   primary_uza_name      3704 non-null   object  
 4   uza_population        4965 non-null   int64   
 5   uza_area_sq_miles     4965 non-null   float64 
 6   year                  4965 non-null   object  
 7   mode                  4965 non-null   object  
 8   type_of_service       4965 non-null   object  
 9   reporter_type         4965 non-null   object  
 10  total_vrh             3964 non-null   float64 
 11  total_upt             3964 non-null   float64 
 12  total_pmt             2432 non-null   float64 
 13  ntd_id_2022           4965 non-null   object  
 14  county                4965 non-null   object  
 15  calt

None

# separate list by both metrics (farebox and funding change)

In [4]:
merge_farebox = ntd_yes_no_merge[ntd_yes_no_merge["metric"]=="Farebox Recovery Ratio"]
merge_funding = ntd_yes_no_merge[ntd_yes_no_merge["metric"]=="Local Funding % Change vs 2019"]

In [5]:
display(
    merge_farebox.shape,
    merge_funding.shape,
    merge_farebox["metric"].unique(),
    merge_funding["metric"].unique(),
    merge_funding.dtypes
)

(2460, 25)

(2505, 25)

array(['Farebox Recovery Ratio'], dtype=object)

array(['Local Funding % Change vs 2019'], dtype=object)

ntd_id                    object
source_agency             object
agency_status             object
primary_uza_name          object
uza_population             int64
uza_area_sq_miles        float64
year                      object
mode                      object
type_of_service           object
reporter_type             object
total_vrh                float64
total_upt                float64
total_pmt                float64
ntd_id_2022               object
county                    object
caltrans_district         object
ntd_entity_name           object
area_type                 object
metric                    object
quartile                  object
metric_short              object
metric_value             float64
requirement               object
requirement_met_flag        bool
_merge                  category
dtype: object

# Group aggregation

## melt big DF 
- so all columns are under 1 column.

In [6]:
group_list_melt = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    "mode",
    "type_of_service",
    "area_type",
    "reporter_type",
    "quartile",
    "metric",
    "metric_value",
    "requirement",
    "requirement_met_flag"
]

value_cols = ["total_upt", "total_vrh", "total_pmt"]

melt_farebox = pd.melt(
    merge_farebox,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

melt_funding = pd.melt(
    merge_funding,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

In [7]:
display(
    melt_farebox.shape,
    melt_funding.shape
)

(7380, 15)

(7515, 15)

In [8]:
sample_ids = ntd_yes_no_merge["ntd_id"].sample(3).to_list()
melt_farebox[melt_farebox["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"]).head()

,source_agency,year,ntd_id,caltrans_district,mode,type_of_service,area_type,reporter_type,quartile,metric,metric_value,requirement,requirement_met_flag,ntd_metric,ntd_metric_value
1155,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Top 25%,Farebox Recovery Ratio,16.01,Met FBR Min,True,total_upt,"6,716.00"
3615,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Top 25%,Farebox Recovery Ratio,16.01,Met FBR Min,True,total_vrh,"2,303.00"
6075,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Top 25%,Farebox Recovery Ratio,16.01,Met FBR Min,True,total_pmt,NaN
1157,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,DR,DO,Rural,Rural Reporter,Top 25%,Farebox Recovery Ratio,16.01,Met FBR Min,True,total_upt,"17,607.00"
3617,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,DR,DO,Rural,Rural Reporter,Top 25%,Farebox Recovery Ratio,16.01,Met FBR Min,True,total_vrh,"6,875.00"


In [9]:
melt_funding[melt_funding["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"]).head()

,source_agency,year,ntd_id,caltrans_district,mode,type_of_service,area_type,reporter_type,quartile,metric,metric_value,requirement,requirement_met_flag,ntd_metric,ntd_metric_value
1196,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Bottom 25%,Local Funding % Change vs 2019,-0.12,Maintained_or_Increased_vs_2019,False,total_upt,"6,716.00"
3701,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Bottom 25%,Local Funding % Change vs 2019,-0.12,Maintained_or_Increased_vs_2019,False,total_vrh,"2,303.00"
6206,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,CB,DO,Rural,Rural Reporter,Bottom 25%,Local Funding % Change vs 2019,-0.12,Maintained_or_Increased_vs_2019,False,total_pmt,NaN
1198,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,DR,DO,Rural,Rural Reporter,Bottom 25%,Local Funding % Change vs 2019,-0.12,Maintained_or_Increased_vs_2019,False,total_upt,"17,607.00"
3703,Morongo Basin Transit Authority (MBTA BUS),2019,91090,8,DR,DO,Rural,Rural Reporter,Bottom 25%,Local Funding % Change vs 2019,-0.12,Maintained_or_Increased_vs_2019,False,total_vrh,"6,875.00"


## aggregation group by
- farebox melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for met FBR YES, per year
    - PMT, UPT, VRH totals for met FBR NO , per year

- funding melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for met funding YES, per year
    - PMT, UPT, VRH totals for met funding NO, per year


# Sanity Check

In [10]:
years = [
    "2020",
    "2021",
    "2022",
    "2023",
    "2024"
]

modes = [
    "MB",
    "CB",
    "RB",
    "TB",
    "DR",
    "VP"
]


In [11]:
melt_farebox[
    (melt_farebox["ntd_id"]=="90013")
    & (melt_farebox["year"].isin(years[:5]))
    & (melt_farebox["mode"].isin(modes))
    & (melt_farebox["ntd_metric"]=="total_upt")
    ].groupby(["year","ntd_id"]).agg(
        ntd_value_mean = ("ntd_metric_value","mean"),
        ntd_value_total = ("ntd_metric_value","sum"),
    ).sort_values(by="year") 
# golden Empire Transit meet/didnt meet FBR requirements through some years. totals and sums still matched
# double checked the NTD TS2.1 report, and these UPT numbers match. 

,,ntd_value_mean,ntd_value_total
year,ntd_id,,
2020,90013,"5,606,491.75","22,425,967.00"
2021,90013,"3,295,866.67","9,887,600.00"
2022,90013,"5,152,039.33","15,456,118.00"
2023,90013,"6,568,710.67","19,706,132.00"
2024,90013,"7,662,590.67","22,987,772.00"


In [12]:
# now introducing requirement flag
melt_farebox[
    (melt_farebox["ntd_id"]=="90004")
    & (melt_farebox["year"].isin(years[:3]))
    & (melt_farebox["mode"].isin(modes))
    & (melt_farebox["ntd_metric"]=="total_upt")
    & (melt_farebox["requirement_met_flag"]==True)
    ].groupby(["year","ntd_id","requirement_met_flag"]).agg(
        ntd_value_mean = ("ntd_metric_value","mean"),
        ntd_value_total = ("ntd_metric_value","sum"),
        ntd_value_median = ("ntd_metric_value","median")
    ).sort_values(by="year") # double checked the NTD TS2.1 report, and these VRH numbers match

,,,ntd_value_mean,ntd_value_total,ntd_value_median
year,ntd_id,requirement_met_flag,,,
2020,90004,True,"2,662,285.50","5,324,571.00","2,662,285.50"
2022,90004,True,"1,600,523.00","3,201,046.00","1,600,523.00"


In [13]:
melt_funding[
    (melt_funding["ntd_id"]=="90013")
    & (melt_funding["mode"].isin(modes))
    & (melt_funding["year"].isin(years))
    & (melt_funding["ntd_metric"]=="total_pmt")
    ].groupby([
        "year",
        "ntd_id",
        # "requirement_met_flag"
    ]).agg(
        ntd_value_mean = ("ntd_metric_value","mean"),
        ntd_value_total = ("ntd_metric_value","sum"),
        ntd_value_median = ("ntd_metric_value","median")).sort_values(by="year")

# NaN values are omitted from mean calculation!!!

,,ntd_value_mean,ntd_value_total,ntd_value_median
year,ntd_id,,,
2020,90013,"29,120,602.75","116,482,411.00","2,575,610.00"
2021,90013,"16,664,939.00","49,994,817.00","1,400,037.00"
2022,90013,"26,995,610.67","80,986,832.00","2,773,895.00"
2023,90013,"31,876,820.00","95,630,460.00","3,438,248.00"
2024,90013,"36,508,636.00","109,525,908.00","3,790,920.00"


## I feel very confident that the aggreagations, averages and sum are working.
notes
- The metrics i queried from the warehouse match the ts2.1 ntd report
- NaN values are not included in the mean calculations
- 

## How many ntd_id are there, how many were True, how many were False

In [14]:
melt_farebox.columns

Index(['source_agency', 'year', 'ntd_id', 'caltrans_district', 'mode',
       'type_of_service', 'area_type', 'reporter_type', 'quartile', 'metric',
       'metric_value', 'requirement', 'requirement_met_flag', 'ntd_metric',
       'ntd_metric_value'],
      dtype='object')

In [18]:
melt_farebox.groupby(
    [
        "metric",
        "area_type",
        "ntd_id",
        "requirement_met_flag",
        "year"
    ]).agg(
    #total_unique_ntd_id =("ntd_id","nunique"),
    true_count = ("requirement_met_flag","count")
)

true_count
metric                 area_type ntd_id requirement_met_flag year            
Farebox Recovery Ratio Rural     90216  False                2020           9
                                                             2021           9
                                                             2022           9
                                                             2023           9
                                                             2024           9
                                        True                 2019           9
                                 91000  False                2019           9
                                                             2020           9
                                                             2021           9
                                                             2022           9
                                                             2023           9
                                                             2024           9
                                 91002  False                2019           3
                                                             2020           3
                                                             2021           3
                                                             2022           3
                                                             2023           3
                                                             2024           3
                                 91005  False                2019          12
                                                             2020          12
                                                             2021          12
                                                             2022          12
                                                             2023          12
                                                             2024          12
                                 91006  False                2020           6
                                                             2021           6
                                                             2022           6
                                                             2023           6
                                                             2024           6
                                        True                 2019           6
                                 91007  False                2021          12
                                        True                 2019          12
                                                             2020          12
                                                             2022          12
                                                             2023          12
                                                             2024          12
                                 91008  False                2021          12
                                                             2022          12
                                                             2023          12
                                                             2024          12
                                 91009  False                2019           9
                                                             2020           9
                                                             2021           9
                                                             2022           9
                                                             2023           9
                                        True                 2024           9
                                 91012  False                2020           9
                                        True                 2019           9
                                                             2021           9
                                                             2022           9
           

## Do yes/no actually match up with the metric value?
- if local funding metric, does metric >=0 actually value = Yes?
- if fbr, does fbr>10 for rural or fbr>20 for urban actually = Yes?

In [ ]:
melt_farebox["metric_value"].astype("int")

In [ ]:
melt_farebox.groupby(
    ["area_type",
    "requirement_met_flag"]
).agg({"ntd_id":"count"})

In [ ]:
melt_farebox[
    (melt_farebox["metric"] == "Farebox Recovery Ratio")
    & (melt_farebox["area_type"]=="Rural")
    & (melt_farebox["metric_value"]>=10)
]

In [ ]:
melt_farebox["sanity_check"] = melt_farebox[
    if area_type == "Urban" & metric_value >= 10:
        "yes"
    else "no"
]